In [ ]:
import random
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


## Generation de dataset

In [5]:
# Listes de descriptions textuelles
distances = ["à 1 mètre", "à 2 mètres", "à 5 mètres", "proche du robot"]
positions = ["devant le robot", "à gauche", "à droite", "derrière le robot"]
états = ["aucune obstruction visible", "un obstacle est détecté", "le robot est bloqué"]

# Générer des commandes en fonction des situations
def generate_textual_dataset(num_samples=1000):
    data = []
    for _ in range(num_samples):
        # Générer une situation
        situation = f"Il y a {random.choice(distances)} {random.choice(positions)}. {random.choice(états)}."
        
        # Définir une commande en fonction d'une règle simple
        if "devant" in situation and "obstacle" in situation:
            commande = "arreter"
        elif "à gauche" in situation and "obstacle" in situation:
            commande = "tourner"
        else:
            commande = "revenir"

        data.append({"situation": situation, "commande": commande})

    return pd.DataFrame(data)

# Génération du dataset
dataset = generate_textual_dataset(1000)
dataset.to_csv("robot_text_commands.csv", index=False)
print("Dataset textuel généré et sauvegardé sous 'robot_text_commands.csv'.")


Dataset textuel généré et sauvegardé sous 'robot_text_commands.csv'.


In [ ]:
# Charger le dataset
dataset = pd.read_csv("robot_text_commands.csv")



In [ ]:
# Extraire les colonnes
texts = dataset["situation"].values
labels = dataset["commande"].values

In [ ]:
# Encodage des étiquettes
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)  # Convertir "tourner", "arrêter", "revenir" en 0, 1, 2

In [ ]:
# Tokenisation des textes
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
# Padding des séquences
max_length = 20  # Longueur maximale des phrases
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

In [ ]:
print(f"Exemple de séquence tokenisée : {sequences[0]}")
print(f"Exemple de séquence après padding : {padded_sequences[0]}")

In [ ]:
# Paramètres
vocab_size = 5000
embedding_dim = 100
input_length = max_length

# Modèle
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes de sortie
])

In [ ]:
# Compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraînement
model.fit(padded_sequences, labels, epochs=10, batch_size=32, validation_split=0.2)